<a href="https://colab.research.google.com/github/TuliDas/Paraphrase-Generation/blob/main/paraphrase_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Paraphrase Generation Using LLM T5 [Text-to-Text Transfer Transformer]


Import Libraries

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, TrainerCallback
import os

# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


**Data Loading and Preporcessing**

In [3]:
dataset = load_dataset("paws" , "labeled_final")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.79k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.43M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [4]:
df = pd.DataFrame(dataset['train'])
df

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
...,...,...,...,...
49396,49397,"`` Our school is of spiritual and spiritual , ...",`` Our School is of the Temporal and the Spiri...,0
49397,49398,She was in Cork on June 24 and arrived on 8 Ju...,"She was at Cork on 24 June , and arrived in th...",1
49398,49399,Cornelia Stuyvesant Vanderbilt ( George and Ed...,John John F. A. Cecil ( the only child of Geor...,0
49399,49400,The third season was premiered on 7 June 2010 ...,"The fourth season was premiered on June 7 , 20...",0


Sentence 1 is input sentence , Sentence 2 is paraphrased sentence.
We don't need any classification here, we only need two sentencces.
'0' represents two sentences are not paraphrased, so we only need label '1' sentences which are paraphrased

In [5]:
df = df[df['label']==1]
df

,id,sentence1,sentence2,label
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
5,6,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",1
7,8,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,1
...,...,...,...,...
49384,49385,"The Romanesque language , Galician ( Galego ) ...",The Romance language currently spoken in Galic...,1
49390,49391,Note that k is a vector consisting of three in...,It is necessary to note that k is a vector con...,1
49393,49394,"Tim Henman won in the final 6 -- 2 , 7 -- 6 , ...","Tim Tim Henman won 6 -- 2 , 7 -- 6 against Yev...",1
49395,49396,He was considered an active member of the coun...,He was considered an active member of the Coun...,1


In [6]:
def preprocess_pows(dataset, label=1):
  df = pd.DataFrame(dataset)
  df = df[df['label'] == label]

  df['input_text'] = "paraphrase :" + df['sentence1']
  df['target_text'] = df['sentence2']
  return df[['input_text' , 'target_text']]

train_df = preprocess_pows(dataset['train']).sample(3000, random_state=42)
test_df = preprocess_pows(dataset['test']).sample(300, random_state=42)
validation_df = preprocess_pows(dataset['validation']).sample(300, random_state=42)


In [7]:
train_df

,input_text,target_text
28667,paraphrase :William died in 1859 and Elizabeth...,"In 1859 , William and Elizabeth died the follo..."
33659,paraphrase :The leaves are generally 1.5-4 mm ...,The leaves are typically 1.5-4 mm wide and 0.2...
17421,paraphrase :Drummer Wayne Bennett left the ban...,"In June 2007 , drummer Wayne Bennett left the ..."
4012,paraphrase :Their skin also secretes chemicals...,"Also , their skin secretes chemicals that are ..."
46074,paraphrase :When the Florentine Republic fell ...,"When the Florentine Republic fell in 1530 , Vo..."
...,...,...
28027,paraphrase :Soon Erko Elblaus and Gertrud Luha...,Erko Elblaus and Gertrud Luhaoja were soon rep...
8711,paraphrase :The first three floors were comple...,The first three floors were completed in 1885 ...
9961,"paraphrase :`` Politically sensitive , economi...","`` Politically sensitive , economically approp..."
46060,paraphrase :He met with a group of Boston 's m...,He met with a group of the most influential mu...


In [8]:
# train,test,validation are in pandas dataset, we need to convert them into dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(validation_df)

train_dataset[0]   #this is a dictionary

{'input_text': 'paraphrase :William died in 1859 and Elizabeth died the following year .',
 'target_text': 'In 1859 , William and Elizabeth died the following year .',
 '__index_level_0__': 28667}

TOKENAIZATION

In [9]:
# we have to download model and tokenaization from hugging face
# initialize tokenizer and model
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
# calculate the maximum length of a input text and token size
[len(text) for text in train_dataset['input_text']]


[72,
 72,
 93,
 102,
 171,
 126,
 122,
 148,
 176,
 160,
 101,
 143,
 92,
 197,
 109,
 143,
 140,
 122,
 83,
 160,
 100,
 138,
 106,
 89,
 135,
 197,
 126,
 133,
 84,
 122,
 115,
 112,
 102,
 99,
 134,
 119,
 160,
 139,
 176,
 108,
 100,
 69,
 137,
 95,
 179,
 119,
 122,
 93,
 112,
 182,
 140,
 170,
 84,
 143,
 121,
 120,
 141,
 118,
 168,
 126,
 95,
 89,
 184,
 126,
 98,
 114,
 168,
 174,
 171,
 70,
 151,
 92,
 142,
 111,
 155,
 108,
 91,
 162,
 69,
 105,
 132,
 65,
 139,
 74,
 181,
 163,
 113,
 121,
 159,
 80,
 164,
 166,
 179,
 150,
 132,
 141,
 138,
 165,
 108,
 120,
 135,
 143,
 87,
 120,
 163,
 110,
 147,
 130,
 156,
 105,
 95,
 191,
 182,
 153,
 178,
 124,
 97,
 167,
 155,
 121,
 142,
 113,
 134,
 184,
 118,
 142,
 122,
 168,
 122,
 111,
 120,
 151,
 90,
 184,
 110,
 126,
 116,
 178,
 107,
 111,
 105,
 92,
 145,
 100,
 129,
 93,
 120,
 174,
 130,
 154,
 139,
 130,
 81,
 100,
 116,
 112,
 50,
 118,
 144,
 65,
 138,
 198,
 140,
 111,
 137,
 137,
 158,
 174,
 84,
 97,
 175,
 108,
 

In [11]:
# this is maximum input text length
max_length = max([len(text) for text in train_dataset['input_text']])
max_length

214

In [12]:
# this is maximum  token length
max_token_length = max([len(tokenizer.encode(text)) for text in train_dataset['input_text']])
max_token_length

76

In [13]:
# tokenization function
# if a sentence have 10 tokens, rest 66 tokens will be assign as '0'
max_length = max([len(tokenizer.encode(text)) for text in train_dataset['input_text']])

def tokenize_function(examples):
  inputs = tokenizer(examples['input_text'] , max_length = max_length , truncation=True , padding="max_length")
  targets = tokenizer(examples['target_text'] , max_length = max_length , truncation=True , padding="max_length")
  inputs['labels'] = targets['input_ids']
  return inputs

#Tokenize dataset
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
validation_dataset = validation_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [14]:
train_dataset[0]

{'input_text': 'paraphrase :William died in 1859 and Elizabeth died the following year .',
 'target_text': 'In 1859 , William and Elizabeth died the following year .',
 '__index_level_0__': 28667,
 'input_ids': [3856,
  27111,
  3,
  10,
  518,
  1092,
  23,
  265,
  3977,
  16,
  507,
  3390,
  11,
  9066,
  3977,
  8,
  826,
  215,
  3,
  5,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


FINE TUNING T5 Model

In [15]:
# set directories in Google Drive

results_dir = "/content/drive/MyDrive/Colab Notebooks/7. Paraphrase Generation/results"
model_dir = "/content/drive/MyDrive/Colab Notebooks/7. Paraphrase Generation/saved_t5_model"

# Create the directories if they don't exists
os.makedirs(results_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir= results_dir,  # Directory where the results (logs, model outputs) will be saved
    overwrite_output_dir=True,  # Overwrite the output directory if it exists
    evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
    eval_steps=500,  # Evaluate every 500 steps (this might not be necessary if evaluation is at epoch level)
    logging_dir='./logs',  # Directory to save logs
    logging_steps=100,  # Log every 100 steps
    learning_rate=3e-5,  # Learning rate for optimizer
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,  # Batch size for evaluation
    num_train_epochs=5,  # Number of epochs to train the model
    weight_decay=0.01,  # L2 regularization to prevent overfitting
    save_strategy="no",  # Disable saving checkpoints during training to save memory
    fp16=True,  # Use mixed precision training to speed up training and reduce memory usage
    report_to="none"  # Disable reporting to avoid unwanted logging to external platforms
)

# Initialize Trainer
trainer = Trainer(
    model=model,  # The model to be trained
    args=training_args,  # The training arguments defined above
    train_dataset=train_dataset,  # The training dataset
    eval_dataset=validation_dataset,  # The validation dataset
)

# Start training
trainer.train()  # Begin the training process


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.257700,0.228758
2,0.226800,0.215845
3,0.198900,0.210818
4,0.190300,0.209941
5,0.185300,0.209723


TrainOutput(global_step=1875, training_loss=0.3099291407267253, metrics={'train_runtime': 436.6588, 'train_samples_per_second': 34.352, 'train_steps_per_second': 4.294, 'total_flos': 1355882803200000.0, 'train_loss': 0.3099291407267253, 'epoch': 5.0})

SAVE MODEL and TOKENIZER

In [17]:
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)


('/content/drive/MyDrive/Colab Notebooks/7. Paraphrase Generation/saved_t5_model/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/7. Paraphrase Generation/saved_t5_model/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/7. Paraphrase Generation/saved_t5_model/spiece.model',
 '/content/drive/MyDrive/Colab Notebooks/7. Paraphrase Generation/saved_t5_model/added_tokens.json')

Load Model and tokenizer

In [18]:
# Load the model and tokenizer from the saved directory
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

Inference (Paraphrase Generation System)

In [19]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the model and tokenizer from the saved directory
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

# Set the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


# Preprocessing function for inference
def preprocess_input(sentence):
    return "paraphrase: " + sentence

# Generate paraphrases with corrected num_beams and num_return_sequences
def generate_paraphrase(input_text, model, tokenizer, max_length=max_length, num_beams=5, num_return_sequences=4, top_k=100, top_p=0.9, temperature=1.0):
    # Preprocess input
    input_text = preprocess_input(input_text)

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=max_length, padding="max_length")

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate paraphrases
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_length + 20,  # Increase max_length for more room in paraphrases
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        top_k=top_k,              # Use top-k sampling for diversity
        top_p=top_p,              # Use top-p sampling for nucleus-based sampling
        temperature=temperature,  # Encourage more exploratory generation
        do_sample=True,           # Enable sampling for top-k and top-p
        early_stopping=True
    )

    # Decode generated outputs
    paraphrased_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return paraphrased_texts

# Example sentence
input_sentence = "The quick brown fox jumps over the lazy dog."

# Generate paraphrases
paraphrased_sentences = generate_paraphrase(
    input_sentence, model, tokenizer,max_length, num_return_sequences=4
)

# Display results
print(f"Original: {input_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")


Original: The quick brown fox jumps over the lazy dog.
Paraphrase 1: The quick brown fox jumps over the lazy dog.
Paraphrase 2: The fast brown fox jumps over the lazy dog.
Paraphrase 3: The quick brown fox leaps over the lazy dog.
Paraphrase 4: Quick brown fox jumps over the lazy dog.


In [20]:
# Example sentence
input_sentence = "She enjoys reading books on rainy afternoons."

# Generate paraphrases
paraphrased_sentences = generate_paraphrase(
    input_sentence, model, tokenizer, num_return_sequences=4
)

# Display results
print(f"Original: {input_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: She enjoys reading books on rainy afternoons.
Paraphrase 1: She enjoys reading books on rainy afternoons.
Paraphrase 2: She enjoys reading books on rainy afternoons .
Paraphrase 3: On rainy afternoons, she enjoys reading books.
Paraphrase 4: On rainy afternoons she enjoys reading books.


In [21]:
# Example sentence
input_sentence = "The dog barked loudly at the stranger outside the house."


# Generate paraphrases
paraphrased_sentences = generate_paraphrase(
    input_sentence, model, tokenizer, num_return_sequences=4
)

# Display results
print(f"Original: {input_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: The dog barked loudly at the stranger outside the house.
Paraphrase 1: The dog barked loudly at the stranger outside the house.
Paraphrase 2: The dog loudly barked at the stranger outside the house.
Paraphrase 3: The dog barked loudly at a stranger outside the house.
Paraphrase 4: The dog shouted loudly at the stranger outside the house.


In [22]:
# Example sentence
input_sentence = "Climate change is one of the most pressing issues of our time."


# Generate paraphrases
paraphrased_sentences = generate_paraphrase(
    input_sentence, model, tokenizer, num_return_sequences=4
)

# Display results
print(f"Original: {input_sentence}")
for i, paraphrase in enumerate(paraphrased_sentences, 1):
    print(f"Paraphrase {i}: {paraphrase}")

Original: Climate change is one of the most pressing issues of our time.
Paraphrase 1: Climate change is one of the most pressing issues of our time.
Paraphrase 2: Climate Change is one of the most pressing issues of our time.
Paraphrase 3: Climate change is one of the most pressing problems of our time.
Paraphrase 4: Climate change is one of the most urgent issues of our time.
